In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

from ngsolve.solvers import *

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)

In [3]:
k = 3
nu = 0.001
dt = 0.001
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=2);
X = FESpace([V,Q])
(u,p), (v,q) = X.TnT()
a = BilinearForm(X)
a += (dt*InnerProduct(u,v) + nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)
      -div(u)*q-div(v)*p)*dx

In [4]:
gfu = GridFunction(X)
velocity = gfu.components[0]
Draw(velocity,mesh,"u",sd=3)
Draw(gfu.components[1],mesh,"p",sd=3)
  

uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Redraw()

In [ ]:
def SimpleNewtonSolve(gfu,a,tol=1e-13,maxits=25):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        du.data = a.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break



In [ ]:
SimpleNewtonSolve(gfu,a)
Draw(gfu.components[1],mesh,"p")
Draw(gfu.components[0],mesh,"u")



In [ ]:
#gfu.vec[:]=0
Newton(a,gfu,freedofs=gfu.space.FreeDofs(),maxit=100,maxerr=1e-11,inverse="umfpack",dampfactor=1,printing=True)

In [5]:
Draw(gfu.components[0],mesh,"u")
dt = 0.001
for i in range(1,10):
    dt = dt*i
    with TaskManager():
        a = BilinearForm(X)
        a += (dt*InnerProduct(u,v) + nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)-div(u)*q-div(v)*p)*dx
        Newton(a,gfu,maxit=200,dampfactor=0.1)
    Redraw()

Newton iteration  0
err =  0.46113970426599427
Newton iteration  1
err =  0.33382061059560086
Newton iteration  2
err =  0.13844401728455924
Newton iteration  3
err =  0.213767448206066
Newton iteration  4
err =  0.17752745294769814
Newton iteration  5
err =  0.10748098406641296
Newton iteration  6
err =  0.04793428040986027
Newton iteration  7
err =  0.01522285585602895
Newton iteration  8
err =  0.003126625987040763
Newton iteration  9
err =  0.00031623706641117287
Newton iteration  10
err =  2.1859584652797786e-08
Newton iteration  11
err =  1.2219742334305981e-15
Newton iteration  0
err =  0.00032136916451385003
Newton iteration  1
err =  0.00028922837619856914
Newton iteration  2
err =  0.00023137668356966906
Newton iteration  3
err =  0.00016195814645869283
Newton iteration  4
err =  9.7171433693381e-05
Newton iteration  5
err =  4.8584234677095344e-05
Newton iteration  6
err =  1.943327092750836e-05
Newton iteration  7
err =  5.829906385889374e-06
Newton iteration  8
err =  1.16